## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carballo,ES,43.2130,-8.6910,61.43,93,5,2.10,clear sky
1,1,Takoradi,GH,4.8845,-1.7554,80.58,80,64,10.29,broken clouds
2,2,Dikson,RU,73.5069,80.5464,18.37,95,100,10.89,light snow
3,3,Nadym,RU,65.5333,72.5167,26.13,92,100,2.84,overcast clouds
4,4,Vardo,NO,70.3705,31.1107,32.11,97,100,10.49,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum desirable temperature for your trip? "))
max_temp = float(input("What is the maximum desirable temperature for your trip? "))

What is the minimum desirable temperature for your trip? 75
What is the maximum desirable temperature for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                  (city_data_df["Max Temp"] >= min_temp)]
ideal_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Takoradi,GH,4.8845,-1.7554,80.58,80,64,10.29,broken clouds
6,6,Georgetown,MY,5.4112,100.3354,80.53,94,20,0.00,few clouds
7,7,Xuddur,SO,4.1213,43.8894,83.55,52,100,11.18,overcast clouds
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,57,40,20.71,scattered clouds
11,11,Bambous Virieux,MU,-20.3428,57.7575,75.45,88,75,11.50,light intensity shower rain
21,21,Black River,JM,18.0264,-77.8487,85.33,56,24,12.86,few clouds
24,24,Juru,MY,5.3120,100.4423,82.38,89,20,2.30,few clouds
27,27,Angoche,MZ,-16.2325,39.9086,80.28,77,0,7.43,clear sky
30,30,Rafaela,AR,-31.2503,-61.4867,75.00,27,0,5.99,clear sky
34,34,Marica,BR,-22.9194,-42.8186,75.15,61,9,17.27,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
ideal_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = ideal_cities_df.dropna()
clean_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Takoradi,GH,80.58,broken clouds,4.8845,-1.7554,
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
7,Xuddur,SO,83.55,overcast clouds,4.1213,43.8894,
10,Arraial Do Cabo,BR,75.16,scattered clouds,-22.9661,-42.0278,
11,Bambous Virieux,MU,75.45,light intensity shower rain,-20.3428,57.7575,
21,Black River,JM,85.33,few clouds,18.0264,-77.8487,
24,Juru,MY,82.38,few clouds,5.3120,100.4423,
27,Angoche,MZ,80.28,clear sky,-16.2325,39.9086,
30,Rafaela,AR,75.00,clear sky,-31.2503,-61.4867,
34,Marica,BR,75.15,clear sky,-22.9194,-42.8186,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel Not Found 404....Skiping")

Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping
Hotel Not Found 404....Skiping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"].astype(bool)].dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Takoradi,GH,80.58,broken clouds,4.8845,-1.7554,Raybow International Hotel
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
7,Xuddur,SO,83.55,overcast clouds,4.1213,43.8894,Banjanay Hotel
10,Arraial Do Cabo,BR,75.16,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
11,Bambous Virieux,MU,75.45,light intensity shower rain,-20.3428,57.7575,Casa Tia Villa
21,Black River,JM,85.33,few clouds,18.0264,-77.8487,Idlers' Rest Beach Hotel
24,Juru,MY,82.38,few clouds,5.3120,100.4423,Starway Hotel
27,Angoche,MZ,80.28,clear sky,-16.2325,39.9086,Guest House HESADA
30,Rafaela,AR,75.00,clear sky,-31.2503,-61.4867,Parra Hotel & Suites
34,Marica,BR,75.15,clear sky,-22.9194,-42.8186,Marica Hills


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
new_dataframe = pd.DataFrame(locations)
new_dataframe

,Lat,Lng
1,4.8845,-1.7554
6,5.4112,100.3354
7,4.1213,43.8894
10,-22.9661,-42.0278
11,-20.3428,57.7575
...,...,...
662,-22.8833,-43.1036
663,-23.8597,35.3472
665,-1.4558,-48.5044
668,-18.5933,-39.7322


In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = new_dataframe
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))